<a href="https://colab.research.google.com/github/Ahmad162539/VMJ/blob/main/newfgsmattack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

import sys
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D  # Updated import statement
from tensorflow.keras.utils import plot_model
from keras import backend as K
from keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/T25 (1).csv')

In [ ]:
set(df['Dataset'].tolist())

In [ ]:
df.columns

In [ ]:
X = [eval(i) for i in df['resized_scaled'].tolist()]
X = np.array(X)
y = np.array(df['SOH'].tolist()) ##%changed

In [ ]:
# labeling for stratified random sampling
label = df['SOH'].tolist()
label_l = []
for cnt in range(len(label)):
    i = label[cnt]
    if i>=1.0 :
        lab = '1~'
        num = 0
    elif 0.98<=i<1.0 :
        lab = '0.98~1'
        num = 1
    elif 0.96<=i<0.98 :
        lab = '0.96~0.98'
        num = 2
    elif 0.94<=i<0.96 :
        lab = '0.94~0.96'
        num = 3
    elif 0.92<=i<0.94 :
        lab = '0.92~0.94'
        num = 4
    elif 0.90<=i<0.92 :
        lab = '0.90~0.92'
        num = 5

    elif 0.88<=i<0.9 :
        lab = '0.88~0.9'
        num = 6
    elif 0.86<=i<0.88 :
        lab = '0.86~0.88'
        num = 7
    elif 0.84<=i<0.86 :
        lab = '0.84~0.86'
        num = 8
    elif 0.82<=i<0.84 :
        lab = '0.82~0.84'
        num = 9
    elif 0.80<=i<0.82 :
        lab = '0.80~0.82'
        num = 10


    elif 0.78<=i<0.8 :
        lab = '0.78~0.8'
        num = 11
    elif 0.76<=i<0.78 :
        lab = '0.76~0.78'
        num = 12
    elif 0.74<=i<0.76 :
        lab = '0.74~0.76'
        num = 13
    elif 0.72<=i<0.74 :
        lab = '0.72~0.74'
        num = 14
    elif 0.70<=i<0.72 :
        lab = '0.70~0.72'
        num = 15

    else:
        lab = '~0.7'
        num = 16
    label_l.append(num)

In [ ]:
sss = StratifiedShuffleSplit(n_splits = 10, test_size = 0.3)
for train_idx, test_idx in sss.split(X, np.array(label_l)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = np.array(y)[train_idx], np.array(y)[test_idx]
    y_valid = np.array(label_l)[test_idx]

In [ ]:
sss = StratifiedShuffleSplit(n_splits = 10, test_size = 0.3)
for train_idx, test_idx1 in sss.split(X_test, y_valid):
    X_valid, X_test_f = X_test[train_idx], X_test[test_idx1]
    y_valid, y_test_f = np.array(y_test)[train_idx], np.array(y_test)[test_idx1]
X_valid.shape

In [ ]:
data_list = []
for i in df['Dataset'][test_idx[test_idx1]].tolist():
    if 'LFP' in i:
        data_list.append('LFP')
    if 'NCA' in i:
        data_list.append('NCA')
    if 'NMC' in i:
        data_list.append('NMC')

In [ ]:
x_train = X_train
x_valid = X_valid
x_test = X_test_f
x_test.shape

In [ ]:
rows = len(X[0])
cols = 3
input_shape = (rows, cols, 1)
x_train = x_train.reshape(x_train.shape[0], rows, cols, 1)
x_test = x_test.reshape(x_test.shape[0], rows, cols, 1)
x_valid = x_valid.reshape(x_valid.shape[0], rows, cols, 1)

x_train = x_train.astype('float32') /255.0
x_test = x_test.astype('float32')/255.0
x_valid = x_valid.astype('float32')/255.0

batch_size = 64
print(x_test.shape)
print(x_train.shape)
print(x_test.shape)

In [ ]:
###
model = Sequential()
model.add(Conv2D(32, kernel_size = (2,2), strides = (1,1), padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(Conv2D(64, kernel_size = (2,2), strides = (1,1), padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1))
model.summary()
###

In [ ]:
start = time.time()
epochs = 5
print('start time : ' , start)
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mse'])
hist2 = model.fit(x_train, y_train, epochs = epochs, verbose = 1, validation_data = (x_valid, y_valid))

In [ ]:
def fgsm_attack(input, epsilon, data_grad):
    # Calculate perturbation by scaling the gradient
    pert_out = input + epsilon * data_grad

    # Clip the perturbation to ensure it doesn't go beyond reasonable values
    pert_out = tf.clip_by_value(pert_out, 0, 1)

    return pert_out

def ifgsm_attack(input, epsilon, data_grad):
    iterations = 10
    alpha = epsilon / iterations
    pert_out = input
    for i in range(iterations):
        # Update perturbation by scaling the gradient
        pert_out = pert_out + alpha * data_grad
        # Clip the perturbation to ensure it doesn't go beyond reasonable values
        pert_out = tf.clip_by_value(pert_out, 0, 1)
    return pert_out

def mifgsm_attack(input, epsilon, data_grad):
    iterations = 10
    decay_factor = 1.0
    pert_out = input
    alpha = epsilon / iterations
    g = 0
    for i in range(iterations):
        # Update gradient with decay and normalize
        g = decay_factor * g + data_grad / tf.norm(data_grad, ord=1)
        # Update perturbation by scaling the gradient
        pert_out = pert_out + alpha * g
        # Clip the perturbation to ensure it doesn't go beyond reasonable values
        pert_out = tf.clip_by_value(pert_out, 0, 1)
    return pert_out

In [ ]:
print(x_test.shape)
print(y_test_f.shape)

Hejran modified based on new code

In [ ]:
def test(model, x_test, y_test_f, epsilon, attack):
    correct = 0
    adv_examples = []

    for idx in range(len(x_test)):
        data, target = x_test[idx], y_test_f[idx]

        # Assuming x_test has shape (batch_size, 128, 3, 1)
        data = tf.convert_to_tensor(data)
        target = tf.convert_to_tensor(target)

        with tf.GradientTape() as tape:
            data = tf.Variable(data, trainable=True)
            output = model(data)
            init_pred = output  # Assuming it's regression, no argmax for regression
            if init_pred.numpy() != target.numpy():
                continue
            loss = tf.keras.losses.mean_squared_error(target, output)

        data_grad = tape.gradient(loss, data)

        if attack == "fgsm":
            perturbed_data = fgsm_attack(data, epsilon, data_grad)
        elif attack == "ifgsm":
            perturbed_data = ifgsm_attack(data, epsilon, data_grad)
        elif attack == "mifgsm":
            perturbed_data = mifgsm_attack(data, epsilon, data_grad)

        perturbed_data = tf.convert_to_tensor(perturbed_data)
        output = model(perturbed_data)
        final_pred = output  # Assuming it's regression, no argmax for regression

        if final_pred.numpy() == target.numpy():
            correct += 1
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.numpy().squeeze()
                adv_examples.append((init_pred.numpy(), final_pred.numpy(), adv_ex))
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.numpy().squeeze()
                adv_examples.append((init_pred.numpy(), final_pred.numpy(), adv_ex))

    final_acc = correct / float(len(x_test))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(x_test), final_acc))

    return final_acc, adv_examples


Chat GPT solved but wrongly

In [ ]:
def test(model, x_test, y_test_f, epsilon, attack):
    correct = 0
    adv_examples = []

    for idx in range(len(x_test)):
        data, target = x_test[idx], y_test_f[idx]

        # Assuming x_test is a list of grayscale images with shape (128, 3, 1)
        data = tf.convert_to_tensor(data, dtype=tf.float32)
        data = tf.expand_dims(data, axis=0)  # Add batch dimension
        target = tf.convert_to_tensor(target, dtype=tf.float32)

        with tf.GradientTape() as tape:
            data = tf.Variable(data, trainable=True)
            output = model(data)
            init_pred = output  # Assuming it's regression, no argmax for regression
            if init_pred.numpy() != target.numpy():
                continue
            loss = tf.keras.losses.mean_squared_error(target, output)

        data_grad = tape.gradient(loss, data)

        if attack == "fgsm":
            perturbed_data = fgsm_attack(data, epsilon, data_grad)
        elif attack == "ifgsm":
            perturbed_data = ifgsm_attack(data, epsilon, data_grad)
        elif attack == "mifgsm":
            perturbed_data = mifgsm_attack(data, epsilon, data_grad)

        perturbed_data = tf.convert_to_tensor(perturbed_data, dtype=tf.float32)
        output = model(perturbed_data)
        final_pred = output  # Assuming it's regression, no argmax for regression

        if final_pred.numpy() == target.numpy():
            correct += 1
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.numpy().squeeze()
                adv_examples.append((init_pred.numpy(), final_pred.numpy(), adv_ex))
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.numpy().squeeze()
                adv_examples.append((init_pred.numpy(), final_pred.numpy(), adv_ex))

    final_acc = correct / float(len(x_test))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(x_test), final_acc))

    return final_acc, adv_examples


In [ ]:
print(x_test.shape)

In [ ]:
def test(model, x_test, y_test_f, epsilon, attack):
    correct = 0
    adv_examples = []

    for idx in range(len(x_test)):
        data, target = x_test[idx], y_test_f[idx]

        data = tf.convert_to_tensor(data, dtype=tf.float32)
        target = tf.convert_to_tensor(target, dtype=tf.float32)

        with tf.GradientTape() as tape:
            data = tf.Variable(data, trainable=True)
            output = model(data)
            init_pred = output  # Assuming it's regression, no argmax for regression
            if init_pred.numpy() != target.numpy():
                continue
            loss = tf.keras.losses.mean_squared_error(target, output)

        data_grad = tape.gradient(loss, data)

        if attack == "fgsm":
            perturbed_data = fgsm_attack(data, epsilon, data_grad)
        elif attack == "ifgsm":
            perturbed_data = ifgsm_attack(data, epsilon, data_grad)
        elif attack == "mifgsm":
            perturbed_data = mifgsm_attack(data, epsilon, data_grad)

        perturbed_data = tf.convert_to_tensor(perturbed_data, dtype=tf.float32)
        output = model(perturbed_data)
        final_pred = output  # Assuming it's regression, no argmax for regression

        if final_pred.numpy() == target.numpy():
            correct += 1
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.numpy().squeeze()
                adv_examples.append((init_pred.numpy(), final_pred.numpy(), adv_ex))
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.numpy().squeeze()
                adv_examples.append((init_pred.numpy(), final_pred.numpy(), adv_ex))

    final_acc = correct / float(len(x_test))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(x_test), final_acc))

    return final_acc, adv_examples


In [ ]:
epsilons = [0, 0.007, 0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3]

for attack in ("fgsm", "ifgsm", "mifgsm"):
    accuracies = []
    examples = []
    for eps in epsilons:
        acc, ex = test(model, x_test, y_test_f, eps, attack)
        accuracies.append(acc)
        examples.append(ex)

    plt.figure(figsize=(5, 5))
    plt.plot(epsilons, accuracies, "*-")
    plt.title(attack)
    plt.xlabel("Epsilon")
    plt.ylabel("Accuracy")
    plt.show()

    cnt = 0
    plt.figure(figsize=(8, 10))
    for i in range(len(epsilons)):
        for j in range(len(examples[i])):
            cnt += 1
            plt.subplot(len(epsilons), len(examples[0]), cnt)
            plt.xticks([], [])
            plt.yticks([], [])
            if j == 0:
                plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
            orig, adv, ex = examples[i][j]
            plt.title("{} -> {}".format(orig, adv))
            plt.imshow(ex, cmap="gray")
    plt.tight_layout()
    plt.show()


In [ ]:
epsilons = [0, 0.007, 0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3]

for attack in ("fgsm", "ifgsm", "mifgsm"):
    accuracies = []
    examples = []
    for eps in epsilons:
        acc, ex = test(model, x_test, y_test_f, eps, attack)
        accuracies.append(acc)
        examples.append(ex)

    plt.figure(figsize=(5, 5))
    plt.plot(epsilons, accuracies, "*-")
    plt.title(attack)
    plt.xlabel("Epsilon")
    plt.ylabel("Accuracy")
    plt.show()

    cnt = 0
    plt.figure(figsize=(8, 10))
    for i in range(len(epsilons)):
        for j in range(len(examples[i])):
            cnt += 1
            plt.subplot(len(epsilons), len(examples[0]), cnt)
            plt.xticks([], [])
            plt.yticks([], [])
            if j == 0:
                plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
            orig, adv, ex = examples[i][j]
            plt.title("{} -> {}".format(orig, adv))
            plt.imshow(ex, cmap="gray")
    plt.tight_layout()
    plt.show()

In [ ]:
y_vloss = hist2.history['val_loss']
y_loss = hist2.history['loss']
x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")
plt.legend()
plt.title('epoch40_loss')

In [ ]:
predict = model.predict(x_test, verbose = 0)

In [ ]:
p = []
t = []
for i in range(len(predict)):
    p.append(predict[i])
    t.append(y_test_f[i])
    #print('predicted SOH: ' , predict[i],',' , 'Answer SOH: '  , y_test[i])

plt.scatter(y_test_f, predict)
plt.plot(y_test_f, y_test_f,c='red')
plt.show()

In [ ]:
r2 = metrics.r2_score(y_test_f, predict)
rmse = metrics.mean_squared_error(y_test_f, predict)**0.5
mse = metrics.mean_squared_error(y_test_f, predict)
mae = metrics.mean_absolute_error(y_test_f, predict)
mape = metrics.mean_absolute_percentage_error(y_test_f,predict)

In [ ]:
dic = {}
dic['r2'] = [r2]
dic['rmse'] = [rmse]
dic['mse'] = [mse]
dic['mae'] = [mae]
dic['mape'] = [mape]
d = pd.DataFrame(dic)
d

In [ ]:
def fgsm_attack(input, epsilon, data_grad):
    # Calculate perturbation by scaling the gradient
    pert_out = input + epsilon * data_grad

    # Clip the perturbation to ensure it doesn't go beyond reasonable values
    pert_out = tf.clip_by_value(pert_out, 0, 1)

    return pert_out

def ifgsm_attack(input, epsilon, data_grad):
    iterations = 10
    alpha = epsilon / iterations
    pert_out = input
    for i in range(iterations):
        # Update perturbation by scaling the gradient
        pert_out = pert_out + alpha * data_grad
        # Clip the perturbation to ensure it doesn't go beyond reasonable values
        pert_out = tf.clip_by_value(pert_out, 0, 1)
    return pert_out

def mifgsm_attack(input, epsilon, data_grad):
    iterations = 10
    decay_factor = 1.0
    pert_out = input
    alpha = epsilon / iterations
    g = 0
    for i in range(iterations):
        # Update gradient with decay and normalize
        g = decay_factor * g + data_grad / tf.norm(data_grad, ord=1)
        # Update perturbation by scaling the gradient
        pert_out = pert_out + alpha * g
        # Clip the perturbation to ensure it doesn't go beyond reasonable values
        pert_out = tf.clip_by_value(pert_out, 0, 1)
    return pert_out


In [ ]:
def fgsm_attack(input, epsilon, data_grad):
    pert_out = input + epsilon * tf.sign(data_grad)
    pert_out = tf.clip_by_value(pert_out, 0, 1)
    return pert_out

def ifgsm_attack(input, epsilon, data_grad):
    iterations = 10
    alpha = epsilon / iterations
    pert_out = input
    for i in range(iterations - 1):
        pert_out = pert_out + alpha * tf.sign(data_grad)
        pert_out = tf.clip_by_value(pert_out, 0, 1)
        if tf.norm(pert_out - input, ord=float('inf')) > epsilon:
            break
    return pert_out

def mifgsm_attack(input, epsilon, data_grad):
    iterations = 10
    decay_factor = 1.0
    pert_out = input
    alpha = epsilon / iterations
    g = 0
    for i in range(iterations - 1):
        g = decay_factor * g + data_grad / tf.norm(data_grad, ord=1)
        pert_out = pert_out + alpha * tf.sign(g)
        pert_out = tf.clip_by_value(pert_out, 0, 1)
        if tf.norm(pert_out - input, ord=float('inf')) > epsilon:
            break
    return pert_out

def test(model, x_test, y_test_f, epsilon, attack):
    correct = 0
    adv_examples = []

    for idx in range(len(x_test)):
        data, target = x_test[idx], y_test_f[idx]

        data = tf.convert_to_tensor(data, dtype=tf.float32)
        target = tf.convert_to_tensor(target, dtype=tf.float32)

        with tf.GradientTape() as tape:
            data = tf.Variable(data, trainable=True)
            output = model(data)
            init_pred = output  # Assuming it's regression, no argmax for regression
            if init_pred.numpy() != target.numpy():
                continue
            loss = tf.keras.losses.mean_squared_error(target, output)

        data_grad = tape.gradient(loss, data)

        if attack == "fgsm":
            perturbed_data = fgsm_attack(data, epsilon, data_grad)
        elif attack == "ifgsm":
            perturbed_data = ifgsm_attack(data, epsilon, data_grad)
        elif attack == "mifgsm":
            perturbed_data = mifgsm_attack(data, epsilon, data_grad)

        perturbed_data = tf.convert_to_tensor(perturbed_data, dtype=tf.float32)
        output = model(perturbed_data)
        final_pred = output  # Assuming it's regression, no argmax for regression

        if final_pred.numpy() == target.numpy():
            correct += 1
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.numpy().squeeze()
                adv_examples.append((init_pred.numpy(), final_pred.numpy(), adv_ex))
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.numpy().squeeze()
                adv_examples.append((init_pred.numpy(), final_pred.numpy(), adv_ex))

    final_acc = correct / float(len(x_test))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(x_test), final_acc))

    return final_acc, adv_examples

In [ ]:

def fgsm_attack(input, epsilon, data_grad):
    pert_out = input + epsilon * tf.sign(data_grad)
    pert_out = tf.clip_by_value(pert_out, 0, 1)
    return pert_out

def ifgsm_attack(input, epsilon, data_grad):
    iterations = 10
    alpha = epsilon / iterations
    pert_out = input
    for i in range(iterations - 1):
        pert_out = pert_out + alpha * tf.sign(data_grad)
        pert_out = tf.clip_by_value(pert_out, 0, 1)
        if tf.norm(pert_out - input, ord=float('inf')) > epsilon:
            break
    return pert_out

def mifgsm_attack(input, epsilon, data_grad):
    iterations = 10
    decay_factor = 1.0
    pert_out = input
    alpha = epsilon / iterations
    g = 0
    for i in range(iterations - 1):
        g = decay_factor * g + data_grad / tf.norm(data_grad, ord=1)
        pert_out = pert_out + alpha * tf.sign(g)
        pert_out = tf.clip_by_value(pert_out, 0, 1)
        if tf.norm(pert_out - input, ord=float('inf')) > epsilon:
            break
    return pert_out


In [ ]:
def test(model, x_test, y_test, epsilon, attack):
    correct = 0
    adv_examples = []

    for idx in range(len(x_test)):
        data, target = x_test[idx], y_test_f[idx]

        data = tf.convert_to_tensor(data, dtype=tf.float32)
        target = tf.convert_to_tensor(target, dtype=tf.float32)

        with tf.GradientTape() as tape:
            data = tf.Variable(data, trainable=True)
            output = model(data)
            init_pred = output  # Assuming it's regression, no argmax for regression
            if init_pred.numpy() != target.numpy():
                continue
            loss = tf.keras.losses.mean_squared_error(target, output)

        data_grad = tape.gradient(loss, data)

        if attack == "fgsm":
            perturbed_data = fgsm_attack(data, epsilon, data_grad)
        elif attack == "ifgsm":
            perturbed_data = ifgsm_attack(data, epsilon, data_grad)
        elif attack == "mifgsm":
            perturbed_data = mifgsm_attack(data, epsilon, data_grad)

        perturbed_data = tf.convert_to_tensor(perturbed_data, dtype=tf.float32)
        output = model(perturbed_data)
        final_pred = output  # Assuming it's regression, no argmax for regression

        if final_pred.numpy() == target.numpy():
            correct += 1
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.numpy().squeeze()
                adv_examples.append((init_pred.numpy(), final_pred.numpy(), adv_ex))
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.numpy().squeeze()
                adv_examples.append((init_pred.numpy(), final_pred.numpy(), adv_ex))

    final_acc = correct / float(len(x_test))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(x_test), final_acc))

    return final_acc, adv_examples


In [ ]:
x_test_reshaped = x_test.reshape(x_test.shape[0], 128, 3, 1)


In [ ]:
epsilons = [0, 0.007, 0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3]

for attack in ("fgsm", "ifgsm", "mifgsm"):
    accuracies = []
    examples = []
    for eps in epsilons:
        acc, ex = test(model, x_test_reshaped, y_test_f, eps, attack)
        accuracies.append(acc)
        examples.append(ex)

    plt.figure(figsize=(5, 5))
    plt.plot(epsilons, accuracies, "*-")
    plt.title(attack)
    plt.xlabel("Epsilon")
    plt.ylabel("Accuracy")
    plt.show()

    cnt = 0
    plt.figure(figsize=(8, 10))
    for i in range(len(epsilons)):
        for j in range(len(examples[i])):
            cnt += 1
            plt.subplot(len(epsilons), len(examples[0]), cnt)
            plt.xticks([], [])
            plt.yticks([], [])
            if j == 0:
                plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
            orig, adv, ex = examples[i][j]
            plt.title("{} -> {}".format(orig, adv))
            plt.imshow(ex, cmap="gray")
    plt.tight_layout()
    plt.show()